### 1. mon_feature.pkl upload


In [1]:
import pandas as pd
import numpy as np
import pickle

FILE_PATH = './content/mon_features.pkl'
LABEL_COLUMN = ['website_label', 'monitored_label']

FEATURES_VER2 = [
    'total_transmission_time', 'std_inter_packet_time',
    'num_outgoing_packets', 'avg_incoming_burst_size',
    'cumul_packets_30pct', 'cumul_packets_10pct',
    'incoming_order_skew', 'outgoing_order_skew',
    'cumul_max', 'bigram_OO', 'avg_outgoing_order_first_30',
    'num_incoming_first_30', 'incoming_packet_ratio'
]

features_df = pd.read_pickle(FILE_PATH)
X = features_df[FEATURES_VER2]
y = features_df[LABEL_COLUMN[0]]

print(X)
print(y)


       total_transmission_time  std_inter_packet_time  num_outgoing_packets  \
0                        10.14               0.041168                 121.0   
1                        10.16               0.163930                  80.0   
2                        11.11               0.066661                 118.0   
3                        13.36               0.047809                 122.0   
4                        10.64               0.038760                 115.0   
...                        ...                    ...                   ...   
18995                    43.91               0.143962                 619.0   
18996                    15.60               0.019465                 552.0   
18997                    14.93               0.016411                 579.0   
18998                    19.91               0.033281                 690.0   
18999                    13.76               0.011074                 757.0   

       avg_incoming_burst_size  cumul_packets_30pct

### 2. Random Forest


Split the dataset into training and testing sets


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Train and test Random Forest


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', n_jobs=-1)
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_test)

print("============ Before Hyperparameter tuning ============")
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_rf)))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


============ Before Hyperparameter tuning ============
RF Accuracy: 0.8330526315789474
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.78      0.78        51
           1       0.95      0.81      0.87        47
           2       0.92      0.96      0.94        48
           3       0.94      0.86      0.90        37
           4       0.86      0.84      0.85        44
           5       0.92      0.88      0.90        56
           6       0.80      0.97      0.88        58
           7       0.80      0.85      0.83        53
           8       0.86      0.78      0.82        49
           9       0.60      0.71      0.65        38
          10       0.80      0.78      0.79        55
          11       0.77      0.82      0.80        45
          12       0.92      0.89      0.91        55
          13       0.75      0.60      0.67        45
          14       0.80      0.77      0.78        52
          15       0.70 

### 3. Hyperparameter tuning by using Grid Search


In [4]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10,20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

search = HalvingGridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1), 
                             param_grid, cv=5, factor=3, n_jobs=-1)
search.fit(X_train, y_train)
y_pred_tuned = search.predict(X_test)

print("============ After Hyperparameter tuning ============")
print("Best Parameters:", search.best_params_)
print("RF Accuracy: {}".format(accuracy_score(y_test, y_pred_tuned)))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))


============ After Hyperparameter tuning ============
Best Parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
RF Accuracy: 0.8370526315789474
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78        51
           1       0.97      0.83      0.90        47
           2       0.92      0.96      0.94        48
           3       0.94      0.86      0.90        37
           4       0.88      0.84      0.86        44
           5       0.92      0.88      0.90        56
           6       0.81      0.97      0.88        58
           7       0.82      0.85      0.83        53
           8       0.88      0.76      0.81        49
           9       0.65      0.74      0.69        38
          10       0.81      0.78      0.80        55
          11       0.81      0.84      0.83        45
          12       0.92      0.89      0.91        55
    